In [2]:
import cv2
import mediapipe as mp
import numpy as np

mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

In [2]:
# Video Feed
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    cv2.imshow('Mediapipe Feed', frame)
    if(cv2.waitKey(10) & 0xFF == ord('q')):
        break

cap.release()
cv2.destroyAllWindows()

## Pose Detection

In [3]:
cap = cv2.VideoCapture('./Dataset/Alldone/22_Eng_C.mp4')
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        # Open cv for some reason defaults to BGR(Blue Green Red) so we convert it to RGB for mediapipe
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = holistic.process(image)
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # FACE
        # mp_drawing.draw_landmarks(image, results.face_landmarks, mp.solutions.face_mesh.FACEMESH_CONTOURS)
        # RIGHT HAND
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
        # LEFT HAND
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
        # POSE
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)

        cv2.namedWindow("Mediapipe Feed", cv2.WINDOW_NORMAL)
        cv2.resizeWindow("Mediapipe Feed", 1280,720)
        cv2.imshow('Mediapipe Feed', image)
        if(cv2.waitKey(10) & 0xFF == ord('q')):
            break   
        # print(mp_holistic)
        # break
    cap.release()
    cv2.destroyAllWindows() 

In [9]:
cap = cv2.VideoCapture('./Dataset/Alldone/9_Kan_C.mp4')
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    # Number of frames to average for smoothing
    num_frames_to_average = 5
    landmark_buffer = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = holistic.process(image)
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Append current frame's landmarks to the buffer
        landmark_buffer.append(results.pose_landmarks)

        # Remove oldest landmarks if the buffer exceeds the specified number of frames
        if len(landmark_buffer) > num_frames_to_average:
            landmark_buffer.pop(0)

        # Calculate average landmark positions over the buffer frames
        avg_landmarks = None
        for landmarks in landmark_buffer:
            if avg_landmarks is None:
                avg_landmarks = landmarks
            else:
                for i, landmark in enumerate(avg_landmarks.landmark):
                    landmark.x += landmarks.landmark[i].x
                    landmark.y += landmarks.landmark[i].y
                    landmark.z += landmarks.landmark[i].z

        if avg_landmarks is not None:
            for landmark in avg_landmarks.landmark:
                landmark.x /= len(landmark_buffer)
                landmark.y /= len(landmark_buffer)
                landmark.z /= len(landmark_buffer)

        # Draw landmarks on the image
        mp_drawing.draw_landmarks(image, avg_landmarks, mp_holistic.POSE_CONNECTIONS)

        cv2.namedWindow("Mediapipe Feed", cv2.WINDOW_NORMAL)
        cv2.resizeWindow("Mediapipe Feed", 1280, 720)
        cv2.imshow('Mediapipe Feed', image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


## Legs detection

In [9]:
import cv2

# Load the pre-trained Haar cascade for lower body detection
lower_body_cascade = cv2.CascadeClassifier('haarcascade_lowerbody.xml')

# Check if the cascade file was loaded successfully
if lower_body_cascade.empty():
    print("Error loading cascade file.")
    exit(1)

# Open the video capture
cap = cv2.VideoCapture('./Dataset/Alldone/9_Kan_C.mp4')  # Use 0 for the default camera, or specify the camera index

while True:
    # Read a frame from the camera
    ret, frame = cap.read()

    if not ret:
        break

    # Convert the frame to grayscale for cascade detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Perform lower body detection
    legs = lower_body_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Draw rectangles around the detected legs
    for (x, y, w, h) in legs:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Display the frame with detected legs
    cv2.imshow('Legs Detection', frame)

    # Exit if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and close the windows
cap.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 

: 

## Line Detection

In [10]:
image = cv2.imread('./Dataset/Alldone/Sample_Pic_Batsman.png')
gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
canimg = cv2.Canny(gray,50,200)

lines = cv2.HoughLines(canimg,1,np.pi/180, 60, np.array([]))

for line in lines:
    rho, theta = line[0]
    a = np.cos(theta)
    b = np.sin(theta)

    x0 = a*rho
    y0 = b*rho

    x1 = int(x0 + 1000*(-b))
    y1 = int(y0 + 1000*(a))
    x2 = int(x0 - 1000*(-b))
    y2 = int(y0 - 1000*(1))
    
    cv2.line(image,(x1,y1),(x2,y2),(0,0,255),2)

cv2.imshow("Lines Detected", image)
cv2.imshow("Canny Detection", canimg)

cv2.waitKey(0)
cv2.destroyAllWindows()

## Find Focal Length